In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
import scipy.stats as stats
import sklearn.linear_model as lm
from sklearn import datasets, linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MultiLabelBinarizer
import sklearn.linear_model as lm
from sklearn import datasets, linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import  metrics


In [17]:
df = pd.read_csv('games.csv', index_col=0)
df = df.drop(['Title', 'Summary', 'Reviews', 'Release Date'], axis=1)
df


,Team,Rating,Times Listed,Number of Reviews,Genres,Plays,Playing,Backlogs,Wishlist
0,"['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']",17K,3.8K,4.6K,4.8K
1,['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",21K,3.2K,6.3K,3.6K
2,"['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",30K,2.5K,5K,2.6K
3,"['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",28K,679,4.9K,1.8K
4,['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",21K,2.4K,8.3K,2.3K
...,...,...,...,...,...,...,...,...,...
1507,['Telltale Games'],3.2,94,94,"['Adventure', 'Point-and-Click']",763,5,223,67
1508,"['Sumo Digital', 'Sega']",2.9,264,264,"['Arcade', 'Racing']",1.5K,49,413,107
1509,['Capcom'],3.7,210,210,"['Brawler', 'RPG']",1.1K,45,487,206
1510,['Larian Studios'],4.1,165,165,"['Adventure', 'RPG', 'Strategy', 'Tactical', '...",269,79,388,602


In [18]:
def NumTransform(x):
    scale = 1 
    if x[-1] == 'K':
        x = x[:-1]
        scale = 1000
    x = float(x)
    return x * scale

df['Number of Reviews'] = df['Number of Reviews'].apply(NumTransform)
df['Plays'] = df['Plays'].apply(NumTransform)
df['Playing'] = df['Playing'].apply(NumTransform)
df['Backlogs'] = df['Backlogs'].apply(NumTransform)
df['Wishlist'] = df['Wishlist'].apply(NumTransform)
df['Times Listed'] = df['Times Listed'].apply(NumTransform)


df

,Team,Rating,Times Listed,Number of Reviews,Genres,Plays,Playing,Backlogs,Wishlist
0,"['Bandai Namco Entertainment', 'FromSoftware']",4.5,3900.0,3900.0,"['Adventure', 'RPG']",17000.0,3800.0,4600.0,4800.0
1,['Supergiant Games'],4.3,2900.0,2900.0,"['Adventure', 'Brawler', 'Indie', 'RPG']",21000.0,3200.0,6300.0,3600.0
2,"['Nintendo', 'Nintendo EPD Production Group No...",4.4,4300.0,4300.0,"['Adventure', 'RPG']",30000.0,2500.0,5000.0,2600.0
3,"['tobyfox', '8-4']",4.2,3500.0,3500.0,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...",28000.0,679.0,4900.0,1800.0
4,['Team Cherry'],4.4,3000.0,3000.0,"['Adventure', 'Indie', 'Platform']",21000.0,2400.0,8300.0,2300.0
...,...,...,...,...,...,...,...,...,...
1507,['Telltale Games'],3.2,94.0,94.0,"['Adventure', 'Point-and-Click']",763.0,5.0,223.0,67.0
1508,"['Sumo Digital', 'Sega']",2.9,264.0,264.0,"['Arcade', 'Racing']",1500.0,49.0,413.0,107.0
1509,['Capcom'],3.7,210.0,210.0,"['Brawler', 'RPG']",1100.0,45.0,487.0,206.0
1510,['Larian Studios'],4.1,165.0,165.0,"['Adventure', 'RPG', 'Strategy', 'Tactical', '...",269.0,79.0,388.0,602.0


In [19]:
print(df.isna().sum())
df.columns
df['Rating'] = df['Rating'].fillna(df['Rating'].mean())
df = df.dropna(axis=0, subset=['Team'])
df['Team'].fillna('[]')
df.isna().sum()

Team                  1
Rating               13
Times Listed          0
Number of Reviews     0
Genres                0
Plays                 0
Playing               0
Backlogs              0
Wishlist              0
dtype: int64


Team                 0
Rating               0
Times Listed         0
Number of Reviews    0
Genres               0
Plays                0
Playing              0
Backlogs             0
Wishlist             0
dtype: int64

In [20]:
import ast
import pandas as pd


mlb = MultiLabelBinarizer()
df['Genres'] = df['Genres'].apply(ast.literal_eval)
df_teams = pd.DataFrame(mlb.fit_transform(df['Team']), columns=mlb.classes_, index=df.index)
df_genres = pd.DataFrame(mlb.fit_transform(df['Genres']), columns=mlb.classes_, index=df.index)
df = df.join(df_teams)
df = df.join(df_genres)
df = df.drop(['Team', 'Genres'], axis=1)
df.tail()

C:\Users\gapli\AppData\Local\Temp\ipykernel_7388\1127498529.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Genres'] = df['Genres'].apply(ast.literal_eval)


,Rating,Times Listed,Number of Reviews,Plays,Playing,Backlogs,Wishlist,,!,"""",...,RPG,Racing,Real Time Strategy,Shooter,Simulator,Sport,Strategy,Tactical,Turn Based Strategy,Visual Novel
1507,3.2,94.0,94.0,763.0,5.0,223.0,67.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1508,2.9,264.0,264.0,1500.0,49.0,413.0,107.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1509,3.7,210.0,210.0,1100.0,45.0,487.0,206.0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1510,4.1,165.0,165.0,269.0,79.0,388.0,602.0,1,0,0,...,1,0,0,0,0,0,1,1,1,0
1511,2.8,184.0,184.0,1700.0,11.0,239.0,73.0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
X = df.drop(['Rating'], axis=1)
y = df['Rating']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
kfold = model_selection.KFold(n_splits=10)


(1359, 107) (1359,)
(152, 107) (152,)


In [28]:


grid = GridSearchCV(make_pipeline(PolynomialFeatures(degree=2), linear_model.ElasticNet(alpha=1.0)),
                    param_grid={'polynomialfeatures__degree': [2],
                        'elasticnet__alpha': [0.1]},
                    cv=kfold,
                    refit=True)

#make_pipeline(PolynomialFeatures(degree=2), linear_model.LinearRegression()).get_params().keys()
grid.fit(X_train, y_train)
print(grid.best_params_)
print("R2: {}".format(metrics.r2_score(y_train, grid.predict(X_train)) ))
print("R2: {}".format(metrics.r2_score(y_test, grid.predict(X_test)) ))

c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.126e+01, tolerance: 3.416e-02
  model = cd_fast.enet_coordinate_descent(
c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.568e+01, tolerance: 3.501e-02
  model = cd_fast.enet_coordinate_descent(
c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.558e+01, tolerance: 3.514e-02
  model = cd_fast.enet_

{'elasticnet__alpha': 0.1, 'polynomialfeatures__degree': 2}
R2: 0.5949138709082918


c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.484e+01, tolerance: 3.855e-02
  model = cd_fast.enet_coordinate_descent(
